In [ ]:
import numpy as np
import pandas as pd
import os
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, matthews_corrcoef, f1_score, precision_score
from numpy import interp
import matplotlib.pyplot as plt
import pickle

In [ ]:
# list of directories

# this_dir = os.path.join(r"C:\Users\yeska\OneDrive - KU Leuven\Documents\master", "thesis\ml")
# data_dir = os.path.join(this_dir, "data")
ml_vars_dir = "/Users/jorismachon/Documents/thesis/ML_data/vars"
# tc_v13_path = os.path.join(data_dir, "tabulated_commits_v13_nov.json")
# tc_v14_path = os.path.join(data_dir, "tabulated_commits_v14_nov.json")
tc_v14_path = "/Users/jorismachon/Documents/thesis/tabulated_commits_v16.csv"
tc_v14_path = "/Users/jorismachon/Documents/thesis/tabulated_commits_v14_nov.json"
jit_data = "/home/deck/Documents/masterGT/mt/jit-vulnerability-prediction-master/scripts/ml_pipeline/data_for_analysis.csv"

## Read this if this is your first time in this notebook

This is not a neat notebook, might be a bit all over the place. Firstly, dont run the cells top down, its probably not gonna work. The first thing you want to run is the imports, and then the directories cell, you only need to change the "this_dir", change it to your own root directory!

### Which cell to run?

Go to section 3. Final function, it contains the cell the defines the main wrapper function. Run that cell defitions, go to the defintion of the children functions and run them as well, go to the defitions of the grandchildren functions and run them, and so on. The cells that defines these functions are a bit all over the place, so here is a quick breakdown of the ones that you need to run:

- Section 1.2 contains def bool_cols_to_bin_cols(df) and def create_time_diff_col(df):
- Section 1.3 contains def balancing(df, desired_ratio)
- section 1.4 contains def feature_selection(X) and def calc_vif(X)
- section 1.6 contains def data_preprocessing(source), this is the main wrapper function as explained in the beginning of section 1. Data pre-processing
- section 2.1.1 contains def tune_hyperparameter(clf, name, cv), it's not used in the final function but you can run it if you want to test it
- section 2.2.1 contains def train_cv_save_results(clf_name, cv, X_final, y_final, save_file_name, Xy_are_np=False) and def save_variables(file_name, variables)
- section 2.3 contains def load_variables(file_name) and def load_show_metrics(file_name)
- section 3 contains def train_ml_from_file(source, clf_name, save_file_name, folds=10, repeats=1), **this is the father of all the functions, this is the one that will be used to produce ml outputs from a source file**.

### File namings for the results

The final function train_ml_from_file(...) receives save_file_name, this is **only** the file_name! not the full path, also make sure that you add the extendion .pkl . The path is already set in the function. Also, please name it as {classifier_name}_k{number of folds}_r{number of repetitions}_{extra settings that you think are important}.pkl . For example, abc_k5_r5_v1.pkl means it uses ada boost classifier (and hence abc), 5 folds, 5 repertions, and version 1 (open up possibilities of other versions if necessary).

## 1. Data pre-processing

1. Deal with null values
2. Non-numerical values
3. X and Y, plus balancing
4. Feature selection with VIF
5. Train, test split based on date
6. **Main function for pre-processing**, this is the most important part if you want to run things (almost) right away. go to this section and run the cell that defines data_preprocessing(...). From that cell, ensure that you run the cells that defines the functions used in data_preprocessing(...), you can simply ctrl+leftclick the function (in vscode) to go the right function definition cell.

### Quick data flattening

tabulated_commits_v13_nov.json has a dictionary in each entry, needed to be flattened. Result is tabulated_commits_v14_nov.json.These cells do not need to be run.

In [ ]:

with open(tc_v13_path, 'r') as f:
    tc_v13 = json.load(f)

In [ ]:
def flatten_json(json_data, parent_key='', sep='_'):
    """
    Flatten nested JSON structure
    """
    items = {}
    for k, v in json_data.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_json(v, new_key, sep=sep))
        else:
            items[new_key] = v
    return items

flattened_data = [flatten_json(item) for item in tc_v13]

In [ ]:
with open(tc_v14_path, 'w') as f:
    json.dump(flattened_data, f, indent=4)

### 1.1. Null values dealing (and init df)
Early pandas stuff, just to get to know the data a bit. There's also a line that replaces the null values with -1

In [ ]:

tc_v14_df = pd.read_json(tc_v14_path)

In [ ]:
tc_v14_df.head()

In [ ]:
# There are actually quite a lot of null values

null_author_x_count = tc_v14_df['author_x'].isnull().sum()
not_null_author_x_count = tc_v14_df['author_x'].notnull().sum()

php_files_not_exist = tc_v14_df['oop_php_files_exist'].isnull().sum() + len(tc_v14_df[tc_v14_df["oop_php_files_exist"]==0])
php_files_exist = len(tc_v14_df[tc_v14_df["oop_php_files_exist"]==1])

print("total data is", len(tc_v14_df))
print("No author count", null_author_x_count)
print("Yes author count", not_null_author_x_count)
print("php files not exist (or not extracted) count", php_files_not_exist)
print("php files exist count", php_files_exist)

Since there's a lot of null values, we still don't know what to do with it, so we're going to replace it with -1 for now

In [ ]:
tc_v14_df_filled = tc_v14_df.copy()
tc_v14_df_filled.fillna(-1, inplace=True)

### 1.2 Non-numerical processings

This is dropping some columns and changing some bool columns to binary. We also make the author creation date a time difference to the commit date.

In [ ]:
# MAIN FUNCTIONS
def bool_cols_to_bin_cols(df):
    bin_cols = ["fix", "is_vulnerable", "neutral", "new_author", "oop_php_files_exist"]
    # Change binaries to 0 and 1
    for col in bin_cols:
        df[col] = df[col].astype(int)

In [ ]:
def print_nonnumeric_cols(df):
    # Identify numeric columns
    numeric_columns = df.select_dtypes(include='number').columns

    # List out non-numeric columns
    non_numeric_columns = df.columns.difference(numeric_columns)

    # Print non-numeric columns
    print("Non-numeric columns:")
    for column in non_numeric_columns:
        print(column)
    

In [ ]:
bin_cols = ["fix", "is_vulnerable", "neutral", "new_author", "oop_php_files_exist"]
# Change binaries to 0 and 1
for col in bin_cols:
    tc_v14_df_filled[col] = tc_v14_df_filled[col].astype(int)

the function run below is to check if there's any nonnumeric cols left

In [ ]:
print_nonnumeric_cols()

This is the better version for the timezone, use it for next iterations!

In [ ]:

# TIMEZONE
tc_v14_df_filled['author_info_created_at'].replace(-1, np.nan, inplace=True)
tc_v14_df_filled['author_date'] = pd.to_datetime(tc_v14_df_filled['author_date'], unit='ms')
tc_v14_df_filled['author_timezone'] = pd.to_timedelta(tc_v14_df_filled['author_timezone'], unit='s')
# Combine author_date and author_timezone to create a new column representing the actual date and time
tc_v14_df_filled['commit_time'] = tc_v14_df_filled['author_date'] + tc_v14_df_filled['author_timezone']

print("length df: ", len(tc_v14_df_filled))
print("number of unique commit SHAs: ", len(tc_v14_df_filled['commit_sha'].unique()))

# columns_to_drop = df.filter(regex='^BoW_').columns
# df_final = df.drop(columns=columns_to_drop)

tc_v14_df_filled['author_info_created_at'] = pd.to_datetime(tc_v14_df_filled['author_info_created_at'], errors='coerce', utc=True)
tc_v14_df_filled['commit_time'] = pd.to_datetime(tc_v14_df_filled['commit_time'], errors='coerce', utc=True)

tc_v14_df_filled.loc[tc_v14_df_filled['author_x'] != None, 'time_difference'] = (tc_v14_df_filled.loc[tc_v14_df_filled['author_x'] != None, 'commit_time'] - tc_v14_df_filled.loc[tc_v14_df_filled['author_x'] != None, 'author_info_created_at']).dt.days
tc_v14_df_filled['time_difference'].fillna(-1, inplace=True)

In [ ]:
def create_time_diff_col(df):
    # TIMEZONE
    df['author_info_created_at'].replace(-1, np.nan, inplace=True)
    df['author_date'] = pd.to_datetime(df['author_date'], unit='ms')
    df['author_timezone'] = pd.to_timedelta(df['author_timezone'], unit='s')
    # Combine author_date and author_timezone to create a new column representing the actual date and time
    df['commit_time'] = df['author_date'] + df['author_timezone']    

    # columns_to_drop = df.filter(regex='^BoW_').columns
    # df_final = df.drop(columns=columns_to_drop)

    df['author_info_created_at'] = pd.to_datetime(df['author_info_created_at'], errors='coerce', utc=True)
    df['commit_time'] = pd.to_datetime(df['commit_time'], errors='coerce', utc=True)

    df.loc[df['author_x'] != None, 'time_difference'] = (df.loc[df['author_x'] != None, 'commit_time'] - df.loc[df['author_x'] != None, 'author_info_created_at']).dt.days
    df['time_difference'].fillna(-1, inplace=True)

In [ ]:
tc_v14_df_filled = tc_v14_df_filled.drop(columns=["appname", "author_info_username", "author_x", "author_y", "commit_sha", "repo", "author_info_created_at", "author_date", "author_timezone"])
print(tc_v14_df_filled)

In [ ]:
print_nonnumeric_cols()

commit time is kept because initially we want to split the data by date, but since we're using repeated fold cross-validation, it's gonna be kinda complicated to do that. This commit_time feature will be removed later on

### 1.3 X and Y, plus balancing

balancing here is undersampling

In [ ]:
y = tc_v14_df_filled['is_vulnerable']
X = tc_v14_df_filled.drop(columns=['is_vulnerable'])
print(X.shape)
print(y.shape)

In [ ]:
#undersampling
desired_ratio = 1 
rus = RandomUnderSampler(sampling_strategy=desired_ratio, random_state=42)
X_balanced, y_balanced = rus.fit_resample(X, y)


In [ ]:
print("X_resampled shape:", X_balanced.shape)
print("y_resampled shape:", y_balanced.shape)

print("vuln count", len(y_balanced[y_balanced==1]))
print("non vuln count", len(y_balanced[y_balanced==0]))

In [ ]:
def balancing(df, desired_ratio):    
    y = df['is_vulnerable']
    X = df.drop(columns=['is_vulnerable'])
    rus = RandomUnderSampler(sampling_strategy=desired_ratio, random_state=42)
    X_balanced, y_balanced = rus.fit_resample(X, y)
    return X_balanced, y_balanced

### 1.4 Feature selection with VIF

VIF removes features that have collinearity, used in palomba jit

In [ ]:
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

def feature_selection(X):
    vif1 = calc_vif(X)
    a=vif1.VIF.max()
    while a > 5:
        maximum_a = vif1.loc[vif1['VIF'] == vif1['VIF'].max()]
        vif1 = vif1.loc[vif1['variables'] != maximum_a.iloc[0,0]]
        vif1 = calc_vif(X[vif1.variables.tolist()])
        a = vif1.VIF.max()
        # print(a)

    X = X[vif1.variables.tolist()]

    return X

In [ ]:
colX = [c for c in feature_selection(X_balanced.drop(columns=['commit_time']))]

In [ ]:
print(colX)
print(len(colX))

In [ ]:
colX.append('commit_time')
X_vif = X_balanced[colX]
print(X_vif.shape)

### 1.5 Train test split with date

this is ended up not used

In [ ]:
def split_based_on_time(X, y, ratio):
    # Sort the DataFrame by commit_time
    data = pd.concat([X, y], axis=1)
    data = data.sort_values(by='commit_time')

    # Calculate the index for splitting
    split_index = int(len(data) * ratio)

    # Split the DataFrame into training and test sets based on the calculated index
    train_data = data.iloc[:split_index]
    test_data = data.iloc[split_index:]

    # Separate X and y for training and test sets
    X_train = train_data.drop(columns=['is_vulnerable'])  # Assuming 'commit_time' is not a feature
    y_train = train_data['is_vulnerable']
    X_test = test_data.drop(columns=['is_vulnerable'])    # Assuming 'commit_time' is not a feature
    y_test = test_data['is_vulnerable']

    return X_train, y_train, X_test, y_test

def train_val_test_split(X, y, ratio):
    X_train, y_train, X_val_test, y_val_test = split_based_on_time(X, y, ratio)
    X_val, y_val, X_test, y_test = split_based_on_time(X_val_test, y_val_test, 0.5)
    return X_train, y_train, X_val, y_val, X_test, y_test


In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(X_vif, y_balanced, 0.6)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

In [ ]:
X_train.drop(columns=["commit_time"], inplace=True)
X_val.drop(columns=["commit_time"], inplace=True)
X_test.drop(columns=["commit_time"], inplace=True)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

In [ ]:
print(len(y_train[y_train==1]))
print(len(y_test[y_test==1]))
print(len(y_val[y_val==1]))

### 1.6 Main pre-processing function

this is the main function that is used in the final function for training machine learning. Its a wrapper for everything.

In [ ]:
def data_preprocessing(source):
    print("Pre-processing data...")
    
    
    # Check if the file ends with .csv or .json
    if source.endswith('.csv'):
        # Read CSV file
        tc_df = pd.read_csv(source)
    elif source.endswith('.json'):
        # Read JSON file
        tc_df = pd.read_json(source)
    else:
        # If the file extension is neither .csv nor .json, raise an error
        raise ValueError("Unsupported file format. Please provide a CSV or JSON file.")
    print("Non-numerical processing")
    tc_df.fillna(-1, inplace=True) #Replace all null values with -1
    
    bool_cols_to_bin_cols(tc_df) #boolean columns to binary
    
    create_time_diff_col(tc_df) #create a time difference col that calculates the time diff between author creation and commit time
    
    tc_df = tc_df.drop(columns=["appname", #drop unused columns
                                                      "author_info_username", 
                                                      "author_x", "author_y", 
                                                      "commit_sha", "repo", 
                                                      "author_info_created_at", 
                                                      "author_date", 
                                                      "author_timezone"])
    print("Balancing...")
    X, y = balancing(tc_df, 1)
    print("Feature selection with VIF...")
    colX = [c for c in feature_selection(X.drop(columns=['commit_time']))]
    X = X[colX]
    print("Pre-processing done!\n")
    return X, y
    

In [ ]:
#test the function
X_final_test, y_final_test = data_preprocessing(tc_v14_path)

In [ ]:
print(X_final_test.shape, y_final_test.shape)
print(X_final_test.values[1427])
print(X_final_np[1427])

## 2. Hyperparameter tuning, training, and show metrics

### 2.1 Hyperparameter tuning

the column commit_time is dropped here (for no good reason, but its not a big deal lol). RandomSearchCV is used for the tuning as it is used in the Palomba paper. Again simply go to the main function for hyperparameter tuning and run it if you want a (semi) plug and play

In [ ]:
X_final = X_vif.drop(columns = ["commit_time"])

In [ ]:
print(X_final.shape, y_balanced.shape)

In [ ]:
#classifiers = [(AdaBoostClassifier(n_estimators=100, random_state=0), "AdaBoostClassifier")]
abc = AdaBoostClassifier(n_estimators=100, random_state=0)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
n_estimators = np.arange(100, 2000, step=100)
learning_rate = [0.001, 0.01, 0.1, 0.2, 0.5]

param_space = {
    "n_estimators": n_estimators,
    "learning_rate": learning_rate
}
random_search = RandomizedSearchCV(abc, param_space, n_iter = 10, cv = cv, scoring='f1', n_jobs=-1, random_state=0, verbose=1)

search = random_search.fit(X_final, y_balanced)

best_param = search.best_params_
best_score = search.best_score_
print(best_param)
print(best_score)

In [ ]:
X_final_np = X_final.values
y_balanced_np = y_balanced.values
print(X_final_np.shape, y_balanced_np.shape)
print("Xfinal is", X_final_np)
print("yfinal is", y_balanced_np)

#### 2.1.1 Hyperparameter tuning: Wrapper function
run this

In [ ]:
# Wrapping function for hyperparam tuning
def tune_hyperparameter(clf, name, cv):    
    if name == "AdaBoostClassifier":
        n_estimators = np.arange(100, 2000, step=100)
        learning_rate = [0.001, 0.01, 0.1, 0.2, 0.5]

        param_space = {
            "n_estimators": n_estimators,
            "learning_rate": learning_rate
        }
    else:
        raise Exception("Hyper parameter tuning param space not set for classifier", name)
    
    random_search = RandomizedSearchCV(clf, param_space, n_iter = 10, cv = cv, scoring='f1', n_jobs=-1, random_state=0, verbose=1)

    search = random_search.fit(X_final, y_balanced)

    best_param = search.best_params_
    best_score = search.best_score_
    print("the best parameters are", best_param)
    print("with (the best) score being", best_score)
    return best_param, best_score

In [ ]:
# test the hyperparam tuning
abc = AdaBoostClassifier(n_estimators=100, random_state=0)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
best_param_test, best_score_test = tune_hyperparameter(abc, "AdaBoostClassifier", cv)

### 2.2 Training the model

the first cell is the first script that was tested to run. The cells after are the wrapper functions

In [ ]:
abc_tuned = AdaBoostClassifier(n_estimators=1900, learning_rate=0.1, random_state=0)
splits_indices = cv.split(X_final_np, y_balanced_np)

mean_fpr = np.linspace(0, 1, 100)
tprs = []
aucs = []

N, P = X_final_np.shape

# Aggregate the importances over folds here:
importances_random = np.zeros(P)

# Loop over crossvalidation folds:
scores = []  # Collect accuracies here

TP = []
FP = []
TN = []
FN = []
tnList = []
fpList = []
fnList = []
tpList = []
precisionList = []
f1List = []
mccList = []

i = 1
count = 0
# for train, test in cv.split(X, y):
train_splits = []
test_splits = []
train_anomaly_percentage = []
test_anomaly_percentage = []
train_anomaly_absolute = []
test_anomaly_absolute = []
counterfold = 1
for train, test in splits_indices:
    print('Fold %s of 10' %counterfold)
    counterfold+=1
    train_splits.append(train)
    test_splits.append(test)
    count += 1

    X_train = X_final_np[train]
    y_train = y_balanced_np[train]
    X_test = X_final_np[test]
    y_test = y_balanced_np[test]

    a, b = np.unique(y_train, return_counts=True)[1]
    train_anomaly_percentage.append(b / (a + b))
    train_anomaly_absolute.append(b)
    c, d = np.unique(y_test, return_counts=True)[1]
    test_anomaly_percentage.append(d / (c + d))
    test_anomaly_absolute.append(d)

    abc_tuned.fit(X_train, y_train)

    # Predict for validation data_raw:

    
    probas_ = abc_tuned.predict_proba(X_test)
    y_pred = abc_tuned.predict(X_test)

    # Compute ROC curve and area under the curve
    
    fpr, tpr, thresholds = roc_curve(y_test, probas_[:, 1], pos_label=1)

    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)

    # calculate confusion matrix, precision, f1 and Matthews Correlation Coefficient

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    precision = precision_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    TN.append(tn)
    TP.append(tp)
    FN.append(fn)
    FP.append(fp)

    tnList.append(tn / (tn + fp))
    tpList.append(tp / (fn + tp))
    fpList.append(fp / (tn + fp))
    fnList.append(fn / (fn + tp))

    precisionList.append(precision)
    f1List.append(f1)
    mccList.append(mcc)

    i += 1
    print("\n")
    
print("confusion matrix")
tnList = 100 * np.array(tnList)
tpList = 100 * np.array(tpList)
fnList = 100 * np.array(fnList)
fpList = 100 * np.array(fpList)
precisionList = 100 * np.array(precisionList)
f1List = 100 * np.array(f1List)
mccList = 100 * np.array(mccList)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
# mean_auc = auc(mean_fpr, mean_tpr)
mean_auc = np.mean(aucs)
std_auc = np.std(aucs)
auc_meanpercent = 100 * mean_auc
auc_stdpercent = 100 * std_auc

variables_to_save = {
    'tprs': tprs,
    'aucs': aucs,
    'N': N,
    'P': P,
    'importances_random': importances_random,
    'scores': scores,
    'TP': TP,
    'FP': FP,
    'TN': TN,
    'FN': FN,
    'tnList': tnList,
    'fpList': fpList,
    'fnList': fnList,
    'tpList': tpList,
    'precisionList': precisionList,
    'f1List': f1List,
    'mccList': mccList,
    'train_splits': train_splits,
    'test_splits': test_splits,
    'train_anomaly_percentage': train_anomaly_percentage,
    'test_anomaly_percentage': test_anomaly_percentage,
    'train_anomaly_absolute': train_anomaly_absolute,
    'test_anomaly_absolute': test_anomaly_absolute
}


#### 2.2.1 Training the model: wrapper functions

the main function is train_cv_save_results(...)

In [ ]:
def save_variables(file_name, variables):
    file_path = os.path.join(ml_vars_dir, file_name)
    with open(file_path, 'wb') as handle:
        pickle.dump(variables, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def train_cv_save_results(clf_name, cv, X_final, y_final, save_file_name, Xy_are_np=False):
    if not Xy_are_np:    
        X_final_np = X_final.values
        y_final_np = y_final.values
    else:
        X_final_np = X_final
        y_final_np = y_final
    if clf_name == "AdaBoostClassifier":
        clf = AdaBoostClassifier(n_estimators=1900, learning_rate=0.1, random_state=0)
    else:
        raise Exception("Clf not set for clf name", clf_name)
    splits_indices = cv.split(X_final_np, y_final_np)

    mean_fpr = np.linspace(0, 1, 100)
    tprs = []
    aucs = []

    N, P = X_final_np.shape

    # Aggregate the importances over folds here:
    importances_random = np.zeros(P)

    # Loop over crossvalidation folds:
    scores = []  # Collect accuracies here

    TP = []
    FP = []
    TN = []
    FN = []
    tnList = []
    fpList = []
    fnList = []
    tpList = []
    precisionList = []
    f1List = []
    mccList = []

    i = 1
    count = 0
    # for train, test in cv.split(X, y):
    train_splits = []
    test_splits = []
    train_anomaly_percentage = []
    test_anomaly_percentage = []
    train_anomaly_absolute = []
    test_anomaly_absolute = []
    counterfold = 1
    for train, test in splits_indices:
        print("Fold-repetition", counterfold)
        counterfold+=1
        train_splits.append(train)
        test_splits.append(test)
        count += 1

        X_train = X_final_np[train]
        y_train = y_final_np[train]
        X_test = X_final_np[test]
        y_test = y_final_np[test]

        a, b = np.unique(y_train, return_counts=True)[1]
        train_anomaly_percentage.append(b / (a + b))
        train_anomaly_absolute.append(b)
        c, d = np.unique(y_test, return_counts=True)[1]
        test_anomaly_percentage.append(d / (c + d))
        test_anomaly_absolute.append(d)

        clf.fit(X_train, y_train)

        # Predict for validation data_raw:

        
        probas_ = clf.predict_proba(X_test)
        y_pred = clf.predict(X_test)

        # Compute ROC curve and area under the curve
        
        fpr, tpr, thresholds = roc_curve(y_test, probas_[:, 1], pos_label=1)

        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)

        # calculate confusion matrix, precision, f1 and Matthews Correlation Coefficient

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        precision = precision_score(y_test, y_pred)
        mcc = matthews_corrcoef(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        TN.append(tn)
        TP.append(tp)
        FN.append(fn)
        FP.append(fp)

        tnList.append(tn / (tn + fp))
        tpList.append(tp / (fn + tp))
        fpList.append(fp / (tn + fp))
        fnList.append(fn / (fn + tp))

        precisionList.append(precision)
        f1List.append(f1)
        mccList.append(mcc)

        i += 1
        print("\n")
        
    tnList = 100 * np.array(tnList)
    tpList = 100 * np.array(tpList)
    fnList = 100 * np.array(fnList)
    fpList = 100 * np.array(fpList)
    precisionList = 100 * np.array(precisionList)
    f1List = 100 * np.array(f1List)
    mccList = 100 * np.array(mccList)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    # mean_auc = auc(mean_fpr, mean_tpr)
    mean_auc = np.mean(aucs)
    std_auc = np.std(aucs)
    auc_meanpercent = 100 * mean_auc
    auc_stdpercent = 100 * std_auc

    variables_to_save = {
        'tprs': tprs,
        'aucs': aucs,
        'N': N,
        'P': P,
        'importances_random': importances_random,
        'scores': scores,
        'TP': TP,
        'FP': FP,
        'TN': TN,
        'FN': FN,
        'tnList': tnList,
        'fpList': fpList,
        'fnList': fnList,
        'tpList': tpList,
        'precisionList': precisionList,
        'f1List': f1List,
        'mccList': mccList,
        'train_splits': train_splits,
        'test_splits': test_splits,
        'train_anomaly_percentage': train_anomaly_percentage,
        'test_anomaly_percentage': test_anomaly_percentage,
        'train_anomaly_absolute': train_anomaly_absolute,
        'test_anomaly_absolute': test_anomaly_absolute,
        'auc_meanpercent': auc_meanpercent,
        'auc_stdpercent' : auc_stdpercent
    }
    save_variables(save_file_name, variables_to_save)
   

In [ ]:
#test train wrapper function
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
train_cv_save_results("AdaBoostClassifier", cv, X_final_test, y_final_test, "abc_k10_r1_v2.pkl")

### 2.3 Load metrics and show

The main function is load_show_metrics(...). Run that and also load_variables(...).

In [ ]:
def load_variables(file_name):
    file_path = os.path.join(ml_vars_dir, file_name)
    with open(file_path, 'rb') as handle:
        loaded_variables = pickle.load(handle)    
    return loaded_variables

In [ ]:
def load_show_metrics(file_name):
    print("Showing metrics for file", file_name)
    # Load variables from the file
    loaded_variables = load_variables(file_name)
    # Return each variable separately
    tprs = loaded_variables['tprs']
    aucs = loaded_variables['aucs']
    N = loaded_variables['N']
    P = loaded_variables['P']
    importances_random = loaded_variables['importances_random']
    scores = loaded_variables['scores']
    TP = loaded_variables['TP']
    FP = loaded_variables['FP']
    TN = loaded_variables['TN']
    FN = loaded_variables['FN']
    tnList = loaded_variables['tnList']
    fpList = loaded_variables['fpList']
    fnList = loaded_variables['fnList']
    tpList = loaded_variables['tpList']
    precisionList = loaded_variables['precisionList']
    f1List = loaded_variables['f1List']
    mccList = loaded_variables['mccList']
    train_splits = loaded_variables['train_splits']
    test_splits = loaded_variables['test_splits']
    train_anomaly_percentage = loaded_variables['train_anomaly_percentage']
    test_anomaly_percentage = loaded_variables['test_anomaly_percentage']
    train_anomaly_absolute = loaded_variables['train_anomaly_absolute']
    test_anomaly_absolute = loaded_variables['test_anomaly_absolute']
    
    mean_auc = np.mean(aucs)
    std_auc = np.std(aucs)
    auc_meanpercent = 100 * mean_auc
    auc_stdpercent = 100 * std_auc
    
    """Show metrics"""
    
    plt.clf()  # Clear the current figure
    
    print("TN: %.02f %% ± %.02f %% - FN: %.02f %% ± %.02f %%" % (np.mean(tnList),
                                                                    np.std(tnList),
                                                                    np.mean(fnList),
                                                                    np.std(fnList)))
    print("FP: %.02f %% ± %.02f %% - TP: %.02f %% ± %.02f %%" % (np.mean(fpList),
                                                                    np.std(fpList),
                                                                    np.mean(tpList),
                                                                    np.std(tpList)))

    print(
        "Precision: %.02f %% ± %.02f %% - F1: %.02f %% ± %.02f %% - MCC: %.02f %% ± %.02f %%" % (np.mean(precisionList),
                                                                                                    np.std(precisionList),
                                                                                                    np.mean(f1List),
                                                                                                    np.std(f1List),
                                                                                                    np.mean(mccList),
                                                                                                    np.std(mccList)))

    print("AUC: %.02f %% ± %.02f %%" % (auc_meanpercent, auc_stdpercent))
    # plt.figure(1)  # Create a new figure
    # plt.plot(mean_fpr, mean_tpr, color='b',
    #             label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
    #             lw=2, alpha=.8)

    # std_tpr = np.std(tprs, axis=0)
    # tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    # tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    # plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
    #                     label=r'$\pm$ 1 std. dev.')

    # xlim = [-0.05, 1.05]
    # ylim = [-0.05, 1.05]
    # plt.xlim(xlim)
    # plt.ylim(ylim)
    # plt.xlabel('False Positive Rate')
    # plt.ylabel('True Positive Rate')
    # plt.legend(loc="lower right")
    # plt.show()


In [ ]:
# test show load metrics wrapper func
load_show_metrics("abc_k10_r1_v1.pkl")
load_show_metrics("abc_k5_r5_v1.pkl")

In [ ]:

"""Show metrics"""

print("TN: %.02f %% ± %.02f %% - FN: %.02f %% ± %.02f %%" % (np.mean(tnList),
                                                                np.std(tnList),
                                                                np.mean(fnList),
                                                                np.std(fnList)))
print("FP: %.02f %% ± %.02f %% - TP: %.02f %% ± %.02f %%" % (np.mean(fpList),
                                                                np.std(fpList),
                                                                np.mean(tpList),
                                                                np.std(tpList)))

print(
    "Precision: %.02f %% ± %.02f %% - F1: %.02f %% ± %.02f %% - MCC: %.02f %% ± %.02f %%" % (np.mean(precisionList),
                                                                                                np.std(precisionList),
                                                                                                np.mean(f1List),
                                                                                                np.std(f1List),
                                                                                                np.mean(mccList),
                                                                                                np.std(mccList)))

print("AUC: %.02f %% ± %.02f %%" % (auc_meanpercent, auc_stdpercent))
plt.figure(1)
plt.clf()

plt.plot(mean_fpr, mean_tpr, color='b',
            label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
            lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                    label=r'$\pm$ 1 std. dev.')

xlim = [-0.05, 1.05]
ylim = [-0.05, 1.05]
plt.xlim(xlim)
plt.ylim(ylim)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")

## 3. Final function

This is the main dish, its the ultra main function that wraps everything. It takes the file, what kind of classifier you want (right now it only supports adaboost), outputfile name, and some cv settings. Run the "def train_ml_from_file(...)" cell, and go to the child functions and run their definition cells as well, and run the children functions of those children functions and so on.

In [ ]:
def train_ml_from_file(source, clf_name, save_file_name, folds=10, repeats=1):
    X_final, y_final = data_preprocessing(source)
    cv = RepeatedStratifiedKFold(n_splits=folds, n_repeats=repeats, random_state=1)
    train_cv_save_results(clf_name, cv, X_final, y_final, save_file_name)
    load_show_metrics(save_file_name)

In [ ]:
train_ml_from_file(tc_v14_path, "AdaBoostClassifier", "abc_k5_r1_test.pkl", folds=5, repeats=1)

## 4. Sanity testing using other datasets

Since the result is suspiciously good, we want to check if the code is faulty or not by testing it against other datasets and compare results. The main code that is tested here is train_cv_save_results(...). The data_preprocessing is not really tested here because well we're using other datasets.

### The JIT palomba dataset

In [ ]:
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from collections import Counter

def calc_vif_jit(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

def feature_selection_jit(data, colX, subset):
    
    X = data[colX]
    vif1 = calc_vif_jit(X)
    a=vif1.VIF.max()
    while a > 5:
        maximum_a = vif1.loc[vif1['VIF'] == vif1['VIF'].max()]
        vif1 = vif1.loc[vif1['variables'] != maximum_a.iloc[0,0]]
        vif1 = calc_vif_jit(X[vif1.variables.tolist()])
        a = vif1.VIF.max()
        # print(a)

    X = data[vif1.variables.tolist()]
    
    return X

def jit_data_preprocessing(subset, oversampling):
    # Read in data_raw and create the variable df to manipulate it
    df = pd.read_csv(jit_data, low_memory=False)

    # remove infinite values and NaN values
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    # variables assignement
    y = df['contributing']
    lb = preprocessing.LabelBinarizer()
    y = lb.fit_transform(y).ravel()
    if subset == 'CK':
        colX = [c for c in feature_selection_jit(df, [c for c in df.columns if "CK_" in c], subset)]
    if subset == 'PROCESS':
        colX = [c for c in feature_selection_jit(df, [c for c in df.columns if "P_" in c], subset)]
    if subset == 'TEXT':
        colX = [c for c in df.columns if "T_" in c]
    if subset == 'PROCESS-TEXT':
        colX = [c for c in feature_selection_jit(df, [c for c in df.columns if "P_" in c], subset)] + [c for c in df.columns if "T_" in c]
    if subset == 'CK-TEXT':
        colX = [c for c in feature_selection_jit(df, [c for c in df.columns if "CK_" in c], subset)] + [c for c in df.columns if "T_" in c]
    if subset == 'CK-PROCESS':
        df['PROCESS_fix'] = df['PROCESS_fix'].astype(int)
        colX = [c for c in feature_selection_jit(df, [c for c in df.columns if "PRODUCT_" in c or "PROCESS_" in c], subset)]
        
    if subset == 'ALL':
        colX = [c for c in feature_selection_jit(df, [c for c in df.columns if "CK_" in c or "P_" in c], subset)] + [c for c in df.columns if "T_" in c]

    groups = np.array(df['project'])

    projects = np.unique(groups)

    projects.sort()
    j = 1
    input_list = []
    target_list = []
    group_list = []
    for i in projects:
        a = df[df.project.isin([i])]
        a = a.sort_index(axis=0, inplace=False)
        a_scaled = a[colX]
        y = a['contributing']
        lb = preprocessing.LabelBinarizer()
        y = lb.fit_transform(y).ravel()


        if oversampling == True:
            if Counter(y)[1] > 1:
                sm = SMOTE(random_state=0, n_jobs=-1, k_neighbors=1)
                a_scaled, y = sm.fit_resample(a_scaled, y)


        group_list.append([i]*len(y))
        target_list.append(y)
        input_list.append(a_scaled)
        j += 1

    X = np.concatenate(input_list)
    y = np.concatenate(target_list)
    groups = np.concatenate(group_list)
    return X, y, groups, colX

In [ ]:
df_jit = pd.read_csv(jit_data, low_memory=False)

In [ ]:

df_jit.head
print("JIT total length", len(df_jit))
print("JIT vuln contributing count", len(df_jit[df_jit["contributing"] == 1]))

In [ ]:
X_jit, y_jit, groups_jit, colX_jit = jit_data_preprocessing('CK-PROCESS', True)
print(X_jit.shape)
print(y_jit.shape)
print(groups_jit.shape)
print('Using: %s' %colX_jit)

In [ ]:
X_jit_osf, y_jit_osf, groups_jit_osf, colX_jit_osf = jit_data_preprocessing('CK-PROCESS', False) #osf means oversampling false
print(X_jit.shape)
print(y_jit.shape)
print(groups_jit.shape)
print('Using: %s' %colX_jit)

In [ ]:

def count_label_occurrences(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    label_counts = dict(zip(unique_labels, counts))
    return label_counts

count_label_occurrences(y_jit_osf)

In [ ]:
cv_jit = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
train_cv_save_results("AdaBoostClassifier", cv_jit, X_jit_osf, y_jit_osf, "jit_process_product_k10_r1_oversamp_false.pkl", True)

In [ ]:
load_show_metrics("jit_process_product_k10_r1_oversamp_true.pkl")

In [ ]:
load_show_metrics("jit_process_product_k10_r1_oversamp_false.pkl")

In [ ]:
desired_ratio = 1 
rus = RandomUnderSampler(sampling_strategy=desired_ratio, random_state=42)
X_jit_osf_bal, y_jit_osf_bal = rus.fit_resample(X_jit_osf, y_jit_osf)

In [ ]:
print(X_jit_osf_bal.shape, y_jit_osf_bal.shape)

In [ ]:
cv_jit = RepeatedStratifiedKFold(n_splits=9, n_repeats=1, random_state=1)
train_cv_save_results("AdaBoostClassifier", cv_jit, X_jit_osf_bal, y_jit_osf_bal, "jit_process_product_k10_r1_oversamp_false_balanced.pkl", True)

In [ ]:
load_show_metrics("jit_process_product_k10_r1_oversamp_false_balanced.pkl")